# Setup

In [11]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy import interpolate

import os
import time

from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import FormatStrFormatter
from matplotlib.animation import FuncAnimation

import seaborn as sns
import json

import h5py as h5 
import pandas as pd

In [2]:
# Enable interactive plot
%matplotlib notebook

In [3]:
from cosmo_bbh_tools import cosmo_bbh_tools as cbt

In [4]:
from astropy.cosmology import WMAP9 as cosmo
import astropy.units as u
H0 = cosmo.H(0)
t0 = 1.0/H0
t0_Myr = t0.decompose().to(u.Myr)
T_HUBBLE_MYR = t0_Myr.value
print(T_HUBBLE_MYR)

14105.485021361645


In [5]:
n_systems_all = [5e1, 5e2, 5e3, 5e4, 5e5, 5e6]
n_systems = n_systems_all[:]
print(n_systems)

zminn = 0.000100 # COMPAS MIN
zmaxx = 0.030000 # COMPAS MAX
Z_MIN_LOG = np.log10(zminn)
Z_MAX_LOG = np.log10(zmaxx)

metallicities = np.logspace(Z_MIN_LOG, Z_MAX_LOG, 10)
print(metallicities)

[50.0, 500.0, 5000.0, 50000.0, 500000.0, 5000000.0]
[0.0001     0.00018847 0.0003552  0.00066943 0.00126166 0.00237782
 0.0044814  0.00844598 0.01591789 0.03      ]


In [6]:
def get_index(n, met):
    assert met < len(metallicities), f"metallicity index cannot exceed {len(metallicities)-1}"
    return n*len(metallicities) + met

# Begin Writing useful information

In [7]:
PathData =     os.environ['WORK'] + '/cosmo_bh_grid/'

In [8]:
Z_ALL, M1_ZAMS_ALL, M2_ZAMS_ALL, M1_CO_ALL, M2_CO_ALL, \
                SMA_CO_ALL, ECC_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, DELAY_TIMES_ALL  = cbt.load_data(n_systems, metallicities, PathData)

loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.00e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.88e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_3.55e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_6.69e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.26e-03_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_2.38e-03_combined.h5
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_4.48e-03_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_8.45e-03_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.59e-02_combin

In [9]:
sfm = cbt.get_sfm(n_systems, metallicities, M1_ZAMS_ALL, M2_ZAMS_ALL)
print(sfm)

Star Forming Mass in 5.00e+01 stars: 2.97e+03 M_sol 

Star Forming Mass in 5.00e+02 stars: 3.19e+04 M_sol 

Star Forming Mass in 5.00e+03 stars: 3.59e+05 M_sol 

Star Forming Mass in 5.00e+04 stars: 3.76e+06 M_sol 

Star Forming Mass in 5.00e+05 stars: 3.75e+07 M_sol 

Star Forming Mass in 5.00e+06 stars: 3.76e+08 M_sol 

[2.97380734e+03 3.19232086e+04 3.59218010e+05 3.75513954e+06
 3.75285972e+07 3.75972741e+08]


In [10]:
# Isolate BBHs
M1_BBH_ALL = cbt.mask_type(M1_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
M2_BBH_ALL = cbt.mask_type(M2_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
SMA_BBH_ALL = cbt.mask_type(SMA_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
ECC_BBH_ALL = cbt.mask_type(ECC_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
TYPE1_BBH_ALL = cbt.mask_type(TYPE1_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
TYPE2_BBH_ALL = cbt.mask_type(TYPE2_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
DELAY_TIMES_BBH_ALL  = cbt.mask_type(DELAY_TIMES_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')

In [11]:
len(M1_BBH_ALL)

60

In [12]:
M1_BBH_METS = cbt.flatten_list_by_met(sfm, metallicities, M1_BBH_ALL)
M2_BBH_METS = cbt.flatten_list_by_met(sfm, metallicities, M2_BBH_ALL)
DELAY_TIMES_METS = cbt.flatten_list_by_met(sfm, metallicities, DELAY_TIMES_BBH_ALL)

# Save raw data

In [13]:
output_dir = "raw_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [14]:
for i in range(len(M1_BBH_ALL)):
    M1_BBH_ALL[i] = list(M1_BBH_ALL[i])

In [15]:
for i in range(len(M2_BBH_ALL)):
    M2_BBH_ALL[i] = list(M2_BBH_ALL[i])

In [16]:
for i in range(len(DELAY_TIMES_BBH_ALL)):
    DELAY_TIMES_BBH_ALL[i] = list(DELAY_TIMES_BBH_ALL[i])

In [31]:
M1_ZAMS_TOTAL = np.zeros(len(M1_ZAMS_ALL))
for i in range(len(M1_ZAMS_ALL)):
    M1_ZAMS_TOTAL[i] = np.sum(M1_ZAMS_ALL[i])
M1_ZAMS_TOTAL = list(M1_ZAMS_TOTAL)   

M2_ZAMS_TOTAL = np.zeros(len(M2_ZAMS_ALL))
for i in range(len(M2_ZAMS_ALL)):
    M2_ZAMS_TOTAL[i] = np.sum(M2_ZAMS_ALL[i])
M2_ZAMS_TOTAL = list(M2_ZAMS_TOTAL)   

In [34]:
M2_ZAMS_TOTAL

[290.3095082273579,
 290.3095082273579,
 290.3095082273579,
 290.3095082273579,
 281.2178826618714,
 291.5153175727272,
 291.5153175727272,
 291.5153175727272,
 288.7794970821875,
 288.7794970821875,
 3017.2571658193224,
 3017.2571658193224,
 3018.1616137787187,
 3024.418344742825,
 3019.9533205815433,
 3020.6746595384257,
 3001.436644397227,
 2995.795779832162,
 3015.5133034269998,
 3015.303527084696,
 33174.245693165096,
 33178.92374669037,
 33212.87561971634,
 33125.97323413142,
 33071.21449748083,
 33023.46232353951,
 33034.32315944454,
 32865.900281845985,
 33034.59291913473,
 33123.71540106221,
 348877.818347045,
 348563.1998279188,
 348467.61369533575,
 348308.468433816,
 348144.6559032222,
 347822.15981840855,
 347752.8948353584,
 347384.3895162193,
 347267.8841327211,
 346442.0677326563,
 3473504.086097972,
 3472959.698064536,
 3473252.1412879205,
 3471937.9002587865,
 3471154.585866553,
 3467613.3919066796,
 3467565.664843322,
 3465175.7658895324,
 3462994.5117307818,
 345498

In [39]:
output_dir = "raw_data"

fname = "M1_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'w') as myfile:
        json.dump(M1_ZAMS_TOTAL, myfile)
        
fname = "M2_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'w') as myfile:
        json.dump(M2_ZAMS_TOTAL, myfile)


In [18]:
import inspect

def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]


In [33]:
for data in [M1_BBH_ALL, M2_BBH_ALL, DELAY_TIMES_BBH_ALL]:
    fname = retrieve_name(data)[0]
    with open(f"{output_dir}/{fname}",'w') as myfile:
        json.dump(data, myfile)

## save M1, M2, Delay times by metallicity

In [20]:
output_dir = "bbh_by_met"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [21]:
fname = f"M1_BBH_METS"
with open(f"{output_dir}/{fname}",'w') as myfile:
    json.dump(M1_BBH_METS,myfile)
    
fname = f"M2_BBH_METS"
with open(f"{output_dir}/{fname}",'w') as myfile:
    json.dump(M2_BBH_METS,myfile)
    
fname = f"DELAY_TIMES_METS"
with open(f"{output_dir}/{fname}",'w') as myfile:
    json.dump(DELAY_TIMES_METS,myfile)

# Save LOGUNIFORM metallicity data

In [20]:
hobbs_pathData        = os.environ['SCRATCH']+f'/supernova_remnant_det/hobbs_combined.h5'

raw_output_dir = "raw_data"

In [13]:
fdata = h5.File(hobbs_pathData, 'r')

In [15]:
M1_CO = fdata['BSE_Double_Compact_Objects']["Mass(1)"][()]
M2_CO = fdata['BSE_Double_Compact_Objects']["Mass(2)"][()]              

TYPE1_CO = fdata['BSE_Double_Compact_Objects']["Stellar_Type(1)"][()]
TYPE2_CO = fdata['BSE_Double_Compact_Objects']["Stellar_Type(2)"][()]

DELAY_TIMES = fdata['BSE_Double_Compact_Objects']["Coalescence_Time"][()]

SEED_CO = fdata['BSE_Double_Compact_Objects']["SEED"][()]

M1_ZAMS = fdata['BSE_System_Parameters']['Mass@ZAMS(1)'][()]
M2_ZAMS = fdata['BSE_System_Parameters']['Mass@ZAMS(2)'][()]

MET1_ZAMS = fdata['BSE_System_Parameters']['Metallicity@ZAMS(1)'][()]
MET2_ZAMS = fdata['BSE_System_Parameters']['Metallicity@ZAMS(2)'][()]

SEED_ZAMS = fdata['BSE_System_Parameters']['SEED'][()]

fdata.close()

In [16]:
data_co = {'M1_CO': M1_CO ,
           'M2_CO': M2_CO,
           'DELAY_TIMES': DELAY_TIMES,
           'TYPE1_CO': TYPE1_CO,
           'TYPE2_CO': TYPE2_CO}
 
# Creates pandas DataFrame.
df_co = pd.DataFrame(data_co, index=SEED_CO)
print(df_co.shape)
df_co.head()

(383264, 5)


,M1_CO,M2_CO,DELAY_TIMES,TYPE1_CO,TYPE2_CO
1654802618,28.157549,22.731678,2.644225e+19,14,14
1654802671,16.407829,14.353583,3.397824e+16,14,14
1654802696,35.709265,35.709265,1.534898e+02,14,14
1654802705,17.182051,20.852162,9.319286e+08,14,14
1654802713,17.851691,16.739270,1.211552e+16,14,14


In [17]:
data_zams = {'M1_ZAMS': M1_ZAMS ,
           'M2_ZAMS': M2_ZAMS,
           'MET1_ZAMS': MET1_ZAMS
          }
 
# Creates pandas DataFrame.
df_zams = pd.DataFrame(data_zams, index=SEED_ZAMS)
print(df_zams.shape)
df_zams.head()

(29107460, 3)


,M1_ZAMS,M2_ZAMS,MET1_ZAMS
1654802569,16.549691,13.634775,0.000438
1654802570,5.396256,3.445196,0.007859
1654802571,8.381638,0.813935,0.002880
1654802572,5.109324,3.235999,0.000635
1654802573,19.019741,11.553273,0.005109


In [18]:
# create a mask to select ZAMS that become DCOs
mask_ZAMS_to_CO = np.in1d(SEED_ZAMS, SEED_CO)
sum(mask_ZAMS_to_CO)

383264

In [26]:
# pick out ZAMS that become DCOs
df_zams_co = df_zams.loc[mask_ZAMS_to_CO]

# combined dataframe with DCOs and ZAMS info
df_merged = pd.concat([df_co, df_zams_co], axis=1)

In [27]:
df_bbh = df_merged[df_merged['TYPE1_CO'] + df_merged['TYPE2_CO'] == 28]
df_bhns = df_merged[df_merged['TYPE1_CO'] + df_merged['TYPE2_CO'] == 27]
df_bns = df_merged[df_merged['TYPE1_CO'] + df_merged['TYPE2_CO'] == 26]

In [38]:
# Separate simulation into metallicity bins
M1_BBH_METS_LOGUNIFORM = []
M2_BBH_METS_LOGUNIFORM = []
DELAY_TIMES_BBH_METS_LOGUNIFORM = []

for i in range(len(met_bin_centers)):
    df_sample = df_bbh[(df_bbh['MET1_ZAMS'] >= met_bin_edges[i]) & (df_bbh['MET1_ZAMS'] <= met_bin_edges[i+1])]
    
    M1_BBH_METS_LOGUNIFORM.append(df_sample['M1_CO'].tolist())
    M2_BBH_METS_LOGUNIFORM.append(df_sample['M2_CO'].tolist())
    DELAY_TIMES_BBH_METS_LOGUNIFORM.append(df_sample['DELAY_TIMES'].tolist())

## write

In [33]:
N_bins = 100
met_bin_edges = np.logspace(np.log10(min(MET1_ZAMS)), np.log10(max(MET1_ZAMS)), N_bins + 1)
met_bin_centers = np.mean(np.vstack([met_bin_edges[0:-1],met_bin_edges[1:]]), axis=0)

met_bin_edges = list(met_bin_edges)  
met_bin_centers = list(met_bin_centers)  

fname = f"met_bin_edges_loguniform"
with open(f"{raw_output_dir}/{fname}",'w') as myfile:
    json.dump(met_bin_edges, myfile)
    
fname = f"met_bin_centers_loguniform"
with open(f"{raw_output_dir}/{fname}",'w') as myfile:
    json.dump(met_bin_centers, myfile)

In [21]:
df_merged.to_csv(f"{raw_output_dir}/DCO_ZAMS_COMBINED_LOGUNIFORM.csv")

In [28]:
df_bbh.to_csv(f"{raw_output_dir}/BBH_ZAMS_COMBINED_LOGUNIFORM.csv")
df_bhns.to_csv(f"{raw_output_dir}/BHNS_ZAMS_COMBINED_LOGUNIFORM.csv")
df_bns.to_csv(f"{raw_output_dir}/BNS_ZAMS_COMBINED_LOGUNIFORM.csv")

In [39]:
met_output_dir = "bbh_by_met"
fname = f"M1_BBH_METS_LOGUNIFORM"
with open(f"{met_output_dir}/{fname}",'w') as myfile:
    json.dump(M1_BBH_METS_LOGUNIFORM,myfile)
    
fname = f"M2_BBH_METS_LOGUNIFORM"
with open(f"{met_output_dir}/{fname}",'w') as myfile:
    json.dump(M2_BBH_METS_LOGUNIFORM,myfile)
    
fname = f"DELAY_TIMES_METS_LOGUNIFORM"
with open(f"{met_output_dir}/{fname}",'w') as myfile:
    json.dump(DELAY_TIMES_BBH_METS_LOGUNIFORM ,myfile)

# Read the existing files

In [7]:
output_dir = "bbh_by_met"

fname = f"M1_BBH_METS"
with open(f"{output_dir}/{fname}",'r') as infile:
    M1_BBH_METS = json.load(infile)
    
fname = f"M2_BBH_METS"
with open(f"{output_dir}/{fname}",'r') as infile:
    M2_BBH_METS = json.load(infile)
    
fname = f"DELAY_TIMES_METS"
with open(f"{output_dir}/{fname}",'r') as infile:
    DELAY_TIMES_METS = json.load(infile)
    
    

output_dir = "raw_data"

fname = f"M1_BBH_ALL"
with open(f"{output_dir}/{fname}",'r') as infile:
    M1_BBH_ALL = json.load(infile)    
    
fname = f"M1_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'r') as infile:
    M1_ZAMS_TOTAL = json.load(infile)   
    
fname = f"M2_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'r') as infile:
    M2_ZAMS_TOTAL = json.load(infile)   
    

In [8]:
sfm = cbt.get_sfm(n_systems, metallicities, M1_ZAMS_TOTAL, M2_ZAMS_TOTAL, output_loc="raw_data/sfm")
print(sfm)

Star Forming Mass in 5.00e+01 stars: 2.97e+03 M_sol 

Star Forming Mass in 5.00e+02 stars: 3.19e+04 M_sol 

Star Forming Mass in 5.00e+03 stars: 3.59e+05 M_sol 

Star Forming Mass in 5.00e+04 stars: 3.76e+06 M_sol 

Star Forming Mass in 5.00e+05 stars: 3.75e+07 M_sol 

Star Forming Mass in 5.00e+06 stars: 3.76e+08 M_sol 

[2.97380734e+03 3.19232086e+04 3.59218010e+05 3.75513954e+06
 3.75285972e+07 3.75972741e+08]


In [24]:
output_dir = "raw_data"
df_merged = pd.read_csv(f"{raw_output_dir}/DCO_ZAMS_COMBINED_LOGUNIFORM.csv", index_col=0)

In [25]:
df_merged

,M1_CO,M2_CO,DELAY_TIMES,TYPE1_CO,TYPE2_CO,M1_ZAMS,M2_ZAMS,MET1_ZAMS
1654802618,28.157549,22.731678,2.644225e+19,14,14,68.296587,56.471188,0.000298
1654802671,16.407829,14.353583,3.397824e+16,14,14,43.763578,35.337091,0.000561
1654802696,35.709265,35.709265,1.534898e+02,14,14,39.800804,39.800804,0.000661
1654802705,17.182051,20.852162,9.319286e+08,14,14,46.892779,32.652745,0.000167
1654802713,17.851691,16.739270,1.211552e+16,14,14,70.706505,65.907013,0.006115
1654802740,16.170930,14.575423,9.515051e+19,14,14,41.537609,35.773010,0.000263
1654802776,14.929267,2.297991,2.746583e+10,14,13,52.938625,20.749990,0.004372
1654802812,22.071850,19.578187,3.668720e+18,14,14,53.797786,48.401335,0.000127
1654802989,17.945494,20.882720,1.678972e+09,14,14,48.140782,34.070552,0.000115
1654803008,11.551990,17.843559,1.176027e+03,14,14,38.533081,26.370344,0.000203
